In [1]:
from flask import Flask, request, send_file, Response # type: ignore
from PIL import Image # type: ignore
from flask_cors import CORS # type: ignore
import io # type: ignore
import tensorflow as tf # type: ignore
import numpy as np # type: ignore
import os # type: ignore
import time # type: ignore

In [2]:
app = Flask(__name__)

# Apply CORS to your Flask app
CORS(app, resources={r"/*": {"origins": "*"}})

# Directory to save processed images
PROCESSED_DIR = "processed Images"

In [3]:

# load the selected TensorFlow model
def load_model(model_path):
    return tf.keras.models.load_model(model_path)


In [4]:

# preprocess the image
def preprocess_image(image):
    ycbcr = image.convert("YCbCr")
    y, cb, cr = ycbcr.split()

    y = tf.keras.preprocessing.image.img_to_array(y)
    y = y.astype("float32") / 255.0
    input_image = y.reshape(1, y.shape[0], y.shape[1], 1)  # Reshape for model input

    return input_image, cb, cr


In [5]:

# postprocess the enhanced image
def postprocess_image(output, cb, cr):
    # Post-processing
    output = output.reshape((output.shape[0], output.shape[1]))
    output *= 255.0
    output = output.clip(0, 255)

    # Create a PIL Image from the enhanced output
    output_image = Image.fromarray(np.uint8(output))

    # Resize the CB and CR channels to match the output size
    cb_resized = cb.resize(output_image.size, Image.BICUBIC)
    cr_resized = cr.resize(output_image.size, Image.BICUBIC)

    # Merge Y, CB, and CR channels to create the final enhanced image
    super_res_image = Image.merge("YCbCr", (output_image, cb_resized, cr_resized))
    super_res_image = super_res_image.convert("RGB")

    return super_res_image


In [6]:
# Function to open a file in the default web browser using only os library
def open_html_file(UI_FilePath):
    if os.name == 'nt':  # For Windows
        os.startfile(UI_FilePath)
    elif os.name == 'posix':  # For Unix-like systems (Linux, macOS, etc.)
        if os.system(f'open "{UI_FilePath}"') != 0:  # macOS
            os.system(f'xdg-open "{UI_FilePath}"')  # Linux
    else:
        print("Unsupported operating system")

In [7]:

# Route to process the uploaded image
@app.route('/process_image', methods=['POST'])
def process_image_route():
    # Load the selected model
    model_path = "./models/" + request.form['model']  # models are stored in a 'models' folder
    model = load_model(model_path)

    # Process the uploaded image
    uploaded_image = request.files['image']
    image = Image.open(uploaded_image)

    # Preprocess the image
    input_image, cb, cr = preprocess_image(image)

    # Capture start time
    start_time = time.time()

    # Use the model to enhance image resolution
    output = model.predict(input_image)[0]

    # Capture end time
    end_time = time.time()
    # Calculate processing time
    processing_time = end_time - start_time

    # Post-process the enhanced image
    output_image = postprocess_image(output, cb, cr)

    # Generate filenames for the original and processed images
    original_filename = uploaded_image.filename
    processed_filename = "enhanced_" + original_filename
    original_filepath = os.path.join(PROCESSED_DIR, original_filename)
    processed_filepath = os.path.join(PROCESSED_DIR, processed_filename)

    # Save the original image
    image.save(original_filepath)

    # Save the processed image
    output_image.save(processed_filepath)

    # Save the processed image to a buffer and send it back to the client
    buffered = io.BytesIO()
    output_image.save(buffered, format="JPEG")
    processed_image_data = buffered.getvalue()

    # Set response headers to indicate that it's an image
    response = Response(processed_image_data, mimetype='image/jpeg')
    response.headers['Content-Disposition'] = 'attachment; filename=processed_image.jpg'

    # Add processing time to response headers
    response.headers['Processing-Time'] = str(processing_time)

    return response


In [8]:

@app.route('/check_server_status', methods=['GET'])
def check_server_status():
    return 'Server is running'

def start_flask():
    # Run the Flask application
    app.run(debug=False)



In [9]:

if __name__ == '__main__':
    UI_FilePath = os.path.abspath("GUI\Web_APP.html")

    # Create the 'processed' directory if it doesn't exist
    if not os.path.exists(PROCESSED_DIR):
        os.makedirs(PROCESSED_DIR)

    # Open the HTML file
    open_html_file(UI_FilePath)

    # Start Flask only when a request is made
    start_flask()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/May/2024 15:30:34] "GET /check_server_status HTTP/1.1" 200 -


1/1 [==============================] - 2s 2s/step


127.0.0.1 - - [30/May/2024 15:30:38] "POST /process_image HTTP/1.1" 200 -
